<a href="https://colab.research.google.com/github/seoyeon7/ML/blob/main/Decision_Trees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [402]:
import math
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np

In [403]:
#iris 데이터셋 불러오기
data = pd.read_csv('/content/iris.data',header=None, names=['sepal_len','sepal_width','petal_len','petal_width','class'])

In [404]:
# 범주형 알고리즘 사용을 위해 데이터 정수화
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(data['sepal_len'])
data["sepal_len"] = encoder.transform(data["sepal_len"])

encoder.fit(data['sepal_width'])
data['sepal_width'] = encoder.transform(data['sepal_width'])

encoder.fit(data['petal_len'])
data['petal_len'] = encoder.transform(data['petal_len'])

encoder.fit(data['petal_width'])
data['petal_width'] = encoder.transform(data['petal_width'])

encoder.fit(data['class'])
data['class'] = encoder.transform(data['class'])

In [405]:
#기술 속성(descriptive features)과 대상 속성 분류(target feature)
index=data[['sepal_len','sepal_width','petal_len','petal_width']]
target = data['class']

In [406]:
# 제대로된 학습을 위한 데이터 셔플링
data = data.sample(frac=1).reset_index(drop=True)

In [407]:
#train set(80%)과 test set(20%)으로 분류
def split(dataset):
    train_size = int(data.shape[0]*0.80)
    test_size = int(data.shape[0]*0.20)
    
    traindata = dataset.iloc[test_size:].reset_index(drop=True)
    testdata = dataset.iloc[:test_size].reset_index(drop=True)
    return traindata,testdata

train = split(data)[0]
test = split(data)[1] 

In [408]:
#엔트로피 계산
def entropy(target_col):
    elements,counts = np.unique(target_col,return_counts = True)
    entropy = -np.sum([(counts[i]/np.sum(counts))*np.log2(counts[i]/np.sum(counts)) for i in range(len(elements))])
    return entropy

In [409]:
# 정보이득 계산

def Gain(data,gain_data,target_name="class"):
    # 전체 데이터 셋에 대한 엔트로피 계산
    entropy_all = entropy(data[target_name])
    
    # 정보이득을 구할 데이터 셋에 대한 엔트로피 계산
    vals,counts= np.unique(data[gain_data],return_counts=True)
    Entropy_tar = np.sum([(counts[i]/np.sum(counts))*entropy(data.where(data[gain_data]==vals[i])) for i in range(len(vals))])
    
    #정보이득 계산
    Infogain = entropy_all - Entropy_tar
    return Infogain

In [410]:
from enum import unique
# ID3 알고리즘 사용
def ID3(data,original,features,target_name="class",parent_node = None):

  # 유일한 값을 가지는 경우 종료 후 속성 반환
  if len(np.unique(data[target_name]))<=1:
    return np.unique(data[target_name])[0]

  # 데이터가 없는 경우 원본 데이터 중 최대 값을 가지는 속성 반환
  elif len(data)==0:
    return np.unique(original[target_name])[np.argmax(np.unique(original[target_name],return_counts=True)[1])]

  # features 가 없을 경우 부모 노드의 속성으로 반환
  elif len(features)==0:
    return parent_node
  
  # 부모 노드의 대상 속성 정의
  else:
    parent_node = np.unique(data[target_name])[np.argmax(np.unique(data[target_name],return_counts=True)[1])]

    # 데이터를 분할할 속성 선택
    values = [Gain(data,feature,target_name) for feature in features]
    best_index = np.argmax(values)
    best_feature = features[best_index]

    # 트리 구조 생성
    tree={best_feature:{}}

    # 최대 정보 이득인 속성 제외
    features = [i for i in features if i !=best_feature]

    # 정보이득이 가장 큰 값에 따라 데이터 분할(sub_data)
    for value in np.unique(data[best_feature]):
      sub_data = data.where(data[best_feature] == value).dropna()

      #sub_data에 따라 IDS 호출하여 sub_data 루트 노드 아래로 하위 트리가 추가됨
      subtree = ID3(sub_data, data, features, target_name, parent_node)
      tree[best_feature][value] = subtree

    return (tree)

In [411]:
#testset으로 예측값 계산
def testset(data,tree):
    queries = data.iloc[:,:-1].to_dict(orient = "records")
    predicted = pd.DataFrame(columns=["predicted"]) 
    
    # 정확도 계산
    for i in range(len(data)):
        predicted.loc[i,"predicted"] = predict(queries[i],tree,1.0) 
    print('prediction accuracy: ',(np.sum(predicted["predicted"] == data["class"])/len(data))*100,'%')

In [412]:
tree = ID3(train,train,train.columns[:-1])
testset(test,tree)

prediction accuracy:  86.66666666666667 %
